In [40]:
import pandas as pd
import numpy as np
import os

# ============================================================================
# LOAD DATA FILES
# ============================================================================
FILE_PATHS = {
    'perf': "Details_Perf_Agences_Mensuel.xlsx",
    'loyalty': "Profils_Clients_Magasins.xlsx",
    'nps': "Satisfaction_Clients_Agences_Trimestres.xlsx",
    'attrib': "Carrefour Voyages Attributes (1).xlsx",
    'compet': "Attributs_Magasins_Concurrents_LECLERC.xlsx",
    'transactions': "Transactions_Magasins_Mensuelles.xlsx"
}

print("="*60)
print("LOADING FILES")
print("="*60)

dfs = {}
for name, path in FILE_PATHS.items():
    if os.path.exists(path):
        try:
            dfs[name] = pd.read_excel(path)
            print(f"✓ {name}: {dfs[name].shape}")
        except Exception as e:
            print(f"✗ {name}: Error - {str(e)}")
    else:
        print(f"✗ {name}: File not found")

# ============================================================================
# RENAME COLUMNS
# ============================================================================
print("\n" + "="*60)
print("RENAMING COLUMNS")
print("="*60)

perf = dfs['perf'].rename(columns={'Agence_code': 'agency_id'})
nps = dfs['nps'].rename(columns={'Agence_Code_CIE': 'agency_id'})
attrib = dfs['attrib'].rename(columns={
    'Agence_Code': 'agency_id',
    'Agence_Code_CIE': 'agency_id_cie',
    'Magasin_Code_1': 'store1',
    'Magasin_Code_2': 'store2'
})
loyalty = dfs['loyalty'].rename(columns={'Magasin_Code_2': 'store2'})
compet = dfs['compet'].rename(columns={'Magasin_Code_1': 'store1'})

print("✓ Columns renamed")

# ============================================================================
# CLEAN ID COLUMNS
# ============================================================================
print("\n" + "="*60)
print("CLEANING ID COLUMNS")
print("="*60)

for df in [perf, nps, attrib, loyalty, compet]:
    for key in ['agency_id', 'agency_id_cie', 'store1', 'store2']:
        if key in df.columns:
            df[key] = df[key].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()

print("✓ IDs cleaned (removed .0 and whitespace)")

# ============================================================================
# MERGE ALL DATASETS
# ============================================================================
print("\n" + "="*60)
print("MERGING DATASETS")
print("="*60)

# Start with attrib as base
merged = attrib.copy()
print(f"Starting: {merged.shape}")

# Merge perf on agency_id
merged = merged.merge(perf, on='agency_id', how='left', suffixes=('', '_perf'))
print(f"+ perf:   {merged.shape}")

# Merge nps on agency_id
merged = merged.merge(nps, on='agency_id', how='left', suffixes=('', '_nps'))
print(f"+ nps:    {merged.shape}")

# Merge loyalty on store2
if 'store2' in merged.columns and 'store2' in loyalty.columns:
    merged = merged.merge(loyalty, on='store2', how='left', suffixes=('', '_loyalty'))
    print(f"+ loyalty: {merged.shape}")

# Merge compet on store1
if 'store1' in merged.columns and 'store1' in compet.columns:
    merged = merged.merge(compet, on='store1', how='left', suffixes=('', '_compet'))
    print(f"+ compet:  {merged.shape}")

# ============================================================================
# SAVE MERGED DATA
# ============================================================================
print("\n" + "="*60)
print("SAVING MERGED DATA")
print("="*60)

merged.to_csv('merged_data.csv', index=False)
print(f"✓ Saved: merged_data.csv")
print(f"  Shape: {merged.shape[0]:,} rows × {merged.shape[1]} columns")
print(f"\nColumns in merged dataset:")
print(merged.columns.tolist())

LOADING FILES
✓ perf: (64637, 14)
✓ loyalty: (2254, 4)
✓ nps: (702, 5)
✓ attrib: (98, 18)
✓ compet: (65, 14)
✓ transactions: (2049, 10)

RENAMING COLUMNS
✓ Columns renamed

CLEANING ID COLUMNS
✓ IDs cleaned (removed .0 and whitespace)

MERGING DATASETS
Starting: (98, 18)
+ perf:   (64606, 31)
+ nps:    (64606, 35)
+ loyalty: (1485938, 38)
+ compet:  (1719917, 51)

SAVING MERGED DATA
✓ Saved: merged_data.csv
  Shape: 1,719,917 rows × 51 columns

Columns in merged dataset:
['agency_id', 'agency_id_cie', 'Agence_Nom', 'Agence_Latitude', 'Agence_Longitude', 'store1', 'store2', 'Magasin_Nom', 'Magasin_Banniere', 'Magasin_Format', 'Magasin_Surface', 'Magasin_Drive', 'Magasin_Adresse_1', 'Magasin_Adresse_2', 'Magasin_Code_Postal', 'Magasin_Ville', 'stoIntDesc', 'stoIntUpdateDate', 'resaYear', 'resaMonthNumber', 'resaMonthYear', 'Agence_nom', 'Region', 'Domaine', 'Destination_Pays', 'Nombre_Resa_Total', 'CA_Total', 'Nombre_Resa_Agence', 'CA_Agence', 'Nombre_Resa_Web', 'CA_Web', 'Trimestre', 'M